In [1]:
import numpy
from matplotlib import pyplot, cm, rcParams
from mpl_toolkits.mplot3d import Axes3D
from numba import jit
%matplotlib inline
rcParams['font.size'] = 16

In [2]:
# problem parameters
nx = 41
ny = 41

l = 1.
h = 1.

dx = l/(nx-1)
dy = h/(ny-1)

l1_target = 1e-6

u = 1

def L1norm(new, old):
    norm = numpy.sum(numpy.abs(new-old))
    return norm

In [26]:
# function to solve using Jacobi
def jacobi(w,ksi,dy,dx,u,l1_target):
    """Iteratively solves the Laplace equation using the Jacobi method
    
    Parameters:
    -----------
    p: 2D array of float
        Initial potential distribution
    l2_target: float
        target for the difference between consecutive solutions
        
    Returns:
    --------
    p: 2D array of float
        Potential distribution after relaxation
    """
    l1norm = 1
    wn = numpy.empty_like(w)
    ksin = numpy.empty_like(ksi)
    iterations = 0
    
    while l1norm > l1_target:
        wn = w.copy()
        ksin = ksi.copy()
        
        w[1:-1,1:-1] = 0.25*(wn[1:-1,2:] + wn[1:-1,:-2] \
                            + wn[2:,1:-1] + wn[:-2,1:-1])
        
        
        ksi[1:-1,1:-1] = 0.25*(ksin[1:-1,2:] + ksin[1:-1,:-2] \
                            + ksin[2:,1:-1] + ksin[:-2,1:-1]) + 1/(4*dx**2)*w[1:-1,1:-1]
        
        # Dirichlet B.C. for ksi, should be automatic
        #ksi[:,0] = 0 #left side
        #ksi[:,-1] = 0 #right side
        #ksi[-1,:] = 0 #top side
        #ksi[0,:] = 0 #bottom side
                               
        # Dirichlet B.C. for w
        w[1:-1,0] = -1/(2*dy**2)*(8*ksi[1:-1,1] - ksi[1:-1,2]) # left side
        w[1:-1,-1] = -1/(2*dy**2)*(8*ksi[1:-1,-2] - ksi[1:-1,-3]) # right side
        w[0,1:-1] = -1/(2*dy**2)*(8*ksi[1,1:-1] - ksi[2,1:-1]) # bottom side                       
        w[-1,1:-1] = -1/(2*dy**2)*(8*ksi[-2,1:-1] - ksi[-3,1:-1]) - 3*u/dy # top side                       
                               
        l1norm = max(L1norm(w,wn),L1norm(ksi,ksin))
        iterations += 1
    print('Converged in {} iterations'.format(iterations))
    return w, ksi

In [27]:
wi = numpy.zeros((nx,ny))
ksii = numpy.zeros((nx,ny))

w, ksi = jacobi(wi.copy(),ksii.copy(),dy,dx,u,l1_target)

Converged in 55 iterations


D:\Program Files\Anaconda\lib\site-packages\ipykernel\__main__.py:29: RuntimeWarning: overflow encountered in multiply
D:\Program Files\Anaconda\lib\site-packages\ipykernel\__main__.py:41: RuntimeWarning: overflow encountered in multiply
D:\Program Files\Anaconda\lib\site-packages\ipykernel\__main__.py:29: RuntimeWarning: invalid value encountered in add


In [28]:
print(ksi[:,1])

[  0.00000000e+000   2.57045572e+093  -5.99681323e+094   3.54099584e+101
  -1.85205329e+108   8.62978706e+114  -3.60116660e+121   1.35237158e+128
  -4.59152140e+134   1.41565567e+141  -3.98124054e+147   1.02589138e+154
  -2.43374011e+160   5.34290574e+166  -1.09167908e+173   2.08938018e+179
  -3.77298987e+185   6.48018831e+191  -1.06778566e+198   1.70310680e+204
  -2.65207467e+210   4.06274767e+216  -6.16032523e+222   9.28659754e+228
  -1.39576136e+235   2.09492159e+241  -3.14252076e+247   4.71300800e+253
  -7.06787312e+259   1.05991447e+266  -1.58946414e+272   2.38358219e+278
  -3.57444990e+284   5.36027384e+290  -8.03816903e+296   1.20526619e+303
              -inf               inf              -inf               inf
   0.00000000e+000]
